In [1]:
import os
import nvtabular as nvt

In [11]:
files = ['/home/jupyter/criteo/criteo_orig/day_0', '/home/jupyter/criteo/criteo_orig/day_1']

In [12]:
dataset = nvt.Dataset(
    path_or_source=files,
    engine='csv'
)

In [13]:
dir(dataset)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bind_dd_method',
 'base_dataset',
 'client',
 'compute',
 'cpu',
 'dtypes',
 'engine',
 'file_partition_map',
 'head',
 'merge',
 'npartitions',
 'num_rows',
 'partition_lens',
 'persist',
 'regenerate_dataset',
 'repartition',
 'sample_dtypes',
 'shuffle_by_keys',
 'tail',
 'to_cpu',
 'to_ddf',
 'to_gpu',
 'to_hugectr',
 'to_iter',
 'to_parquet',
 'validate_dataset']

In [14]:
dataset.sample_dtypes()

1\t5\t110\t\t16\t\t1\t0\t14\t7\t1\t\t306\t\t62770d79\te21f5d58\tafea442f\t945c7fcf\t38b02748\t6fcd6dcb\t3580aa21\t28808903\t46dedfa6\t2e027dc1\t0c7c4231\t95981d1f\t00c5ffb7\tbe4ee537\t8a0b74cc\t4cdc3efa\td20856aa\tb8170bba\t9512c20b\tc38e2f28\t14f65a5d\t25b1b089\td7c1fc0b\t7caf609c\t30436bfc\ted10571d    object
dtype: object

In [20]:
dataset.head()

,1\t5\t110\t\t16\t\t1\t0\t14\t7\t1\t\t306\t\t62770d79\te21f5d58\tafea442f\t945c7fcf\t38b02748\t6fcd6dcb\t3580aa21\t28808903\t46dedfa6\t2e027dc1\t0c7c4231\t95981d1f\t00c5ffb7\tbe4ee537\t8a0b74cc\t4cdc3efa\td20856aa\tb8170bba\t9512c20b\tc38e2f28\t14f65a5d\t25b1b089\td7c1fc0b\t7caf609c\t30436bfc\ted10571d
0,0\t32\t3\t5\t\t1\t0\t0\t61\t5\t0\t1\t3157\t5\t...
1,0\t\t233\t1\t146\t1\t0\t0\t99\t7\t0\t1\t3101\t...
2,0\t\t24\t\t11\t24\t\t0\t56\t3\t\t2\t20456\t\t\...
3,0\t60\t223\t6\t15\t5\t0\t0\t1\t8\t0\t2\t1582\t...
4,1\t7\t1\t10\t2\t\t3\t0\t0\t12\t1\t\t4\t2\t8a2b...
